BDLE 2021

date du document  :  13/10/2021 9:00

# PROJET DM1 : IMDB et YAGO (avec COLAB)


Mini-projet sur la restructuration de données et l'analyse de données.
Voir le moodle pour la date de remise et éventuelles précisions sur le travail à faire.
Concernant la forme attendue de vos commentaires: les cellules de texte vous permettent une mise en forme claire et soignée avec la syntaxe markdown.


## Préparation

Vérifier que des ressources de calcul sont allouées à votre notebook est connecté (cf RAM  de disque indiqués en haut à droite) . Sinon cliquer sur le bouton connecter pour obtenir des ressources.




Pour accéder directement aux fichiers stockées sur votre google drive. Renseigner le code d'authentification lorsqu'il est demandé

Ajuster le nom de votre dossier : MyDrive/ens/bdle/DM1

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

drive_dir = "/content/drive/MyDrive/ens/bdle/DM1"
os.makedirs(drive_dir, exist_ok=True)
os.listdir(drive_dir)

Mounted at /content/drive


['csvfiles_sample001',
 'csvfiles_sample001.tgz',
 'name.csv.gz',
 'title.csv.gz',
 'yago']

Installer pyspark et findspark :


In [ ]:
!pip install -q pyspark
!pip install -q findspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 65.7 MB/s 


Démarrer la session spark

In [ ]:
import os
# !find /usr/local -name "pyspark"
os.environ["SPARK_HOME"] = "/usr/local/lib/python3.7/dist-packages/pyspark"
os.environ["JAVA_HOME"] = "/usr"

In [ ]:
# Principaux import
import findspark
from pyspark.sql import SparkSession 
from pyspark import SparkConf  

# pour les dataframe et udf
from pyspark.sql import *  
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *

# pour le chronomètre
import time

# initialise les variables d'environnement pour spark
findspark.init()

# Démarrage session spark 
# --------------------------
def demarrer_spark():
  local = "local[*]"
  appName = "TP"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "6G").\
  set("spark.driver.memory","6G").\
  set("spark.sql.catalogImplementation","in-memory")
  
  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")

  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (4 coeurs)
  spark.conf.set("spark.sql.shuffle.partitions","4")    
  print("session démarrée, son id est ", sc.applicationId)
  return spark
spark = demarrer_spark()

session démarrée, son id est  local-1639079484242


In [ ]:
# on utilise 8 partitions au lieu de 200 par défaut
#spark.conf.set("spark.sql.shuffle.partitions", "8")
print("Nombre de partitions utilisées : ", spark.conf.get("spark.sql.shuffle.partitions"))

Nombre de partitions utilisées :  4


In [ ]:
# Optionnel :
# pour l'accès à spark UI : voir https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip ngrok-stable-linux-amd64.zip
# get_ipython().system_raw('./ngrok http 4050 &')
# !curl -s http://localhost:4040/api/tunnels

Redéfinir la fonction **display** pour afficher le resutltat des requêtes dans un tableau

In [ ]:
import pandas as pd
from google.colab import data_table

# alternatives to Databricks display function.

def display(df, n=100):
  return data_table.DataTable(df.limit(n).toPandas(), include_index=False, num_rows_per_page=10)

def display2(df, n=20):
  pd.set_option('max_columns', None)
  pd.set_option('max_colwidth', None)
  return df.limit(n).toPandas().head(n)

Définir le tag **%%sql** pour pouvoir écrire plus simplement des requêtes en SQL dans une cellule

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic, register_line_cell_magic)

def removeComments(query):
  result = ""
  for line in query.split('\n'):
    if not(line.strip().startswith("--")):
      result += line + " "
  return result

@register_line_cell_magic
def sql(line, cell=None):
    "To run a sql query. Use:  %%sql"
    val = cell if cell is not None else line
    tabRequetes = removeComments(val).split(";")
    for r in tabRequetes:
        if len(r.strip()) > 2:
          derniere = spark.sql(r)
    return display(derniere)

In [ ]:
# facultatif (à ne pas utiliser)
# %load_ext google.colab.data_table
# %unload_ext google.colab.data_table

## Accès aux données

URL pour l'accès aux datasets

In [ ]:
# URL du dossier PUBLIC_DATASET contenant des fichiers de données pour les TP
# ---------------------------------------------------------------------------
# en cas de problème avec le téléchargement des datasets, aller directement sur l'URL ci-dessous
PUBLIC_DATASET_URL = "https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4" 
PUBLIC_DATASET=PUBLIC_DATASET_URL + "/download?path="

print("URL pour les datasets ", PUBLIC_DATASET_URL)

URL pour les datasets  https://nuage.lip6.fr/s/H3bpyRGgnCq2NR4


#### IMDB
Télécharger les fichiers de IMDB directement sur votre drive, sans passer par votre ordinateur personnel.

##### Sample: échantillon de la base IMDB

In [ ]:
import os
from urllib import request
import tarfile

imdb_dir = drive_dir

url = PUBLIC_DATASET + "/imdb/vldb2015/csvfiles_sample001.tgz"
# temp = "/temp"
local_file = imdb_dir + "/csvfiles_sample001.tgz"

os.makedirs(imdb_dir, exist_ok=True)

if(os.path.isfile(local_file)):
  print("file is already downloaded")
else:
  print("downloading from URL: ", url, "save in : ", local_file)
  request.urlretrieve(url, local_file)
  os.chdir(imdb_dir)
  tar = tarfile.open(local_file)
  tar.extractall()
  tar.close()

# Liste des fichiers de IMDB
dir = imdb_dir + "/csvfiles_sample001/"
os.listdir(dir)

file is already downloaded


['title.csv',
 'cast_info.csv',
 'movie_info.csv',
 'person_info.csv',
 'name.csv',
 'movie_companies.csv',
 'company_name.csv',
 'info_type.csv',
 'role_type.csv',
 'kind_type.csv',
 'company_type.csv']

##### Données complètes: name_all et title_all
Télécharger le fichier complet des noms de personnes et les titres de films

In [ ]:
import os
def load_IMDB_file(file):
  if(os.path.isfile( imdb_dir + "/" + file)):
    print(file, "is already stored")
  else:
    url = PUBLIC_DATASET + "/imdb/vldb2015/csvfiles/" + file
    print("downloading from URL: ", url, "save in : " + imdb_dir + "/" + file)
    request.urlretrieve(url , imdb_dir + "/"  + file)

load_IMDB_file("name.csv.gz")
load_IMDB_file("title.csv.gz")

# Liste des fichiers de IMDB
os.listdir(imdb_dir)

name.csv.gz is already stored
title.csv.gz is already stored


['csvfiles_sample001',
 'csvfiles_sample001.tgz',
 'name.csv.gz',
 'title.csv.gz',
 'yago']

In [ ]:
# name_all

schema_name = """
  id int,
  name string,
  imdb_index string,
  imdb_id int,
  gender string,
  name_pcode_cf string,
  name_pcode_nf string,
  surname_pcode string,
  md5sum string
"""

name_all = spark.read.csv(path = imdb_dir + "/name.csv.gz", schema = schema_name)
name_all.createOrReplaceTempView("name_all")
display(name_all)

,id,name,imdb_index,imdb_id,gender,name_pcode_cf,name_pcode_nf,surname_pcode,md5sum
0,3343,"Abela, Mike",None,NaN,m,A1452,M214,A14,61f733c0298a7cb5a461fd787a655d70
1,446,"A., David",None,NaN,m,A313,D13,A,cf45e7b42fbc800c61462988ad1156d2
2,126,"-Alverio, Esteban Rodriguez",None,NaN,m,A4162,E2315,A416,f5c410bff6839b545d04c531f776e8f2
3,1678,"Abbas, Athar",None,NaN,m,A1236,A3612,A12,cf230f6ed718a330dc688c20b8d741d3
4,3610,"Aberer, Leo",None,NaN,m,A164,L16,A16,05684f16b84db387e613d3de57eca5b7
...,...,...,...,...,...,...,...,...,...
95,3211,"Abeghraz, Rachid",None,NaN,m,A1262,R2312,None,2b054445d528ca9f6994adedd9db9722
96,2129,"Abbott, Percy",None,NaN,m,A1316,P6213,A13,7992faa0fc9acd53bebbb68549f9360a
97,1167,"Abacan, Jose Mari",None,NaN,m,A1252,J2561,A125,06f8d8a8e7df8e7887ea7e7c7a671b8c
98,331,"5, Channel",None,NaN,m,C54,None,None,4db55ab39bb71cbe53eda4eacb768705


In [ ]:
# title_all

schema_title = """
          id INT, 
          title STRING, 
          imdb_index STRING, 
          kind_id INT, 
          production_year INT, 
          imdb_id INT, 
          phonetic_code STRING, 
          episode_id STRING,
          season_nr INT, 
          episode_nr INT, 
          series_years STRING, 
          md5sum STRING
        """
title_all = spark.read.csv(path = imdb_dir + "/title.csv.gz", schema = schema_title)
title_all.createOrReplaceTempView("title_all")
display(title_all)

,id,title,imdb_index,kind_id,production_year,imdb_id,phonetic_code,episode_id,season_nr,episode_nr,series_years,md5sum
0,80889,(#1.66),None,7,1980.0,NaN,None,80815,1.0,66.0,None,4de45f35edf0b753c54ab72dcbe68bb5
1,5156,Josie Duggar's 1st Shoes,None,7,2010.0,NaN,J2326,5022,4.0,13.0,None,8d492850166137b550ceb0e9a5b46086
2,197772,(#2.8),None,7,1962.0,NaN,None,197751,2.0,8.0,None,152b0f7213dd467760823b0d6387a6ec
3,111913,(2012-09-13),None,7,2012.0,NaN,None,111095,NaN,NaN,None,daf144630ef1ab1cd5d1d028b406cdcf
4,117556,(#1.1042),None,7,NaN,NaN,None,117506,1.0,1042.0,None,be62adfea75e456d47ddab5c68ac96b0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,74371,American Justice,None,2,2004.0,NaN,A5625,None,NaN,NaN,2004-????,d0d9d994eb34142614894127dc98f750
96,5392,(#1.8),None,7,2011.0,NaN,None,5383,1.0,8.0,None,66aa6123117ad570cff22a0961cde934
97,115268,The Boy Next Door,None,7,2008.0,NaN,B5236,114979,13.0,18.0,None,2da5276828def251a95a367c5a74359a
98,120056,(#2.9),None,7,2009.0,NaN,None,120036,2.0,9.0,None,a44636910a50bf5613640b4c80ff3e37


#### YAGO
Télécharger les fichiers de YAGO

In [ ]:
YAGO_dir = drive_dir + "/yago"

os.makedirs(YAGO_dir, exist_ok=True)

def yagoLoad(file):
  url = PUBLIC_DATASET + "/yago/yago4_2020/" + file
  local_file = YAGO_dir + "/" + file

  if(os.path.isfile( YAGO_dir + "/" + file)):
    print(file, "is already stored ")
  else:
    print("downloading from URL: ", url, "save in : ", local_file)
    request.urlretrieve(url, local_file)
    
yagoLoad("yago-actor-facts.snappy.orc")
yagoLoad("yago-person.snappy.orc")

# Liste des fichiers de YAGO
os.listdir(YAGO_dir)

yago-actor-facts.snappy.orc is already stored 
yago-person.snappy.orc is already stored 


['yago-actor-facts.snappy.orc', 'yago-person.snappy.orc']

##### Les personnes dans Yago

In [ ]:
yagoPerson = spark.read.orc(path = YAGO_dir + "/yago-person.snappy.orc")
yagoPerson.createOrReplaceTempView("yagoPerson")
display(yagoPerson)

,subject
0,<http://yago-knowledge.org/resource/Heywood_L....
1,<http://yago-knowledge.org/resource/Ramanichan...
2,<http://yago-knowledge.org/resource/Yegor_Kiry...
3,<http://yago-knowledge.org/resource/Patrick_J....
4,<http://yago-knowledge.org/resource/Basilio_J....
...,...
95,<http://yago-knowledge.org/resource/Billy_Gowers>
96,<http://yago-knowledge.org/resource/Simon_Kain...
97,<http://yago-knowledge.org/resource/K._Sukumaran>
98,<http://yago-knowledge.org/resource/Harry_Melr...


##### Les faits des acteurs dans Yago

In [ ]:
actorFacts = spark.read.orc(path = YAGO_dir + "/yago-actor-facts.snappy.orc")
actorFacts.createOrReplaceTempView("actorFacts")
display(actorFacts)

,subject,property,object
0,<http://yago-knowledge.org/resource/40_(record...,<http://schema.org/alumniOf>,<http://yago-knowledge.org/resource/Academy_of...
1,<http://yago-knowledge.org/resource/40_(record...,<http://schema.org/nationality>,<http://yago-knowledge.org/resource/Canada>
2,<http://yago-knowledge.org/resource/40_(record...,<http://schema.org/birthPlace>,<http://yago-knowledge.org/resource/Toronto>
3,<http://yago-knowledge.org/resource/40_(record...,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Actor>
4,<http://yago-knowledge.org/resource/40_(record...,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Songwriter>
...,...,...,...
95,<http://yago-knowledge.org/resource/Aage_Fønss>,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Opera_singer>
96,<http://yago-knowledge.org/resource/Aage_Fønss>,<http://schema.org/hasOccupation>,<http://yago-knowledge.org/resource/Actor>
97,<http://yago-knowledge.org/resource/Aage_Fønss>,<http://schema.org/birthDate>,"""1887-12-12""^^<http://www.w3.org/2001/XMLSchem..."
98,<http://yago-knowledge.org/resource/Aage_Fønss>,<http://schema.org/deathPlace>,<http://yago-knowledge.org/resource/Copenhagen>


## Définir les tables IMDB

Le dossier contenant les fichiers csv de IMDB

In [ ]:
dir = drive_dir + "/csvfiles_sample001/"

### Les tables initiales

#### Title
La table Title affecte un identifiant id à un film

In [ ]:
schema_title = """
          id INT, 
          title STRING, 
          imdb_index STRING, 
          kind_id INT, 
          production_year INT, 
          imdb_id INT, 
          phonetic_code STRING, 
          episode_id STRING,
          season_nr INT, 
          episode_nr INT, 
          series_years STRING, 
          md5sum STRING
        """

title = spark.read.csv(path = dir + "title.csv", schema = schema_title).persist()
title.createOrReplaceTempView("Title")
# display(title)




#### Kind_Type et Info_Type
* La table Kind_type
* La table Info_type indique la nature d'un attribut d'information. 

Sert dans Movie_Info, Person_Info

In [ ]:
#============
# Kind_Type
#============
schema_kind_type = "id INT, kind STRING"

kind_type = spark.read.csv(path = dir + "kind_type.csv", schema = schema_kind_type).persist()

kind_type.createOrReplaceTempView("Kind_Type")
# kind_type.printSchema()
#display(kind_type)



#==============
# Info_Type
#==============
schema_info_type = "id INT, info STRING"

info_type = spark.read.csv(path = dir + "info_type.csv", schema = schema_info_type).persist()

info_type.createOrReplaceTempView("Info_Type")
#info_type.printSchema()

display(info_type)

,id,info
0,1,runtimes
1,2,color info
2,3,genres
3,4,languages
4,5,certificates
...,...,...
95,96,other literature
96,97,mpaa
97,98,plot
98,99,votes distribution


#### Movie_info

In [ ]:
#==============
# Movie_Info
#==============
schema_movie_info = """
    id int,
    movie_id int,
    info_type_id int,
    info string,
    note string
"""

movie_info = spark.read.csv(path = dir + "movie_info.csv", schema = schema_movie_info).persist()

movie_info.createOrReplaceTempView("Movie_Info")
#movie_info.printSchema()

display(movie_info)

,id,movie_id,info_type_id,info,note
0,11658949,13,16,Italy:29 March 2012,None
1,11659887,434,16,Netherlands:7 January 2013,None
2,11661227,1680,16,USA:11 May 2011,None
3,11661228,1680,16,USA:11 May 2011,None
4,11661771,1841,16,Spain:20 March 2005,None
...,...,...,...,...,...
95,11671027,10280,16,UK:25 February 1984,None
96,11671080,10331,16,UK:11 February 1984,None
97,11671391,10750,16,USA:29 August 2009,None
98,11672626,11564,16,Australia:14 July 2012,None


#### Cast_Info

In [ ]:
#==============
# Cast_Info
#==============
schema_cast_info = """
  id int,
  person_id int,
  movie_id int,
  person_role_id int,
  note string,
  nr_order int,
  role_id int
"""

cast_info = spark.read.csv(path = dir + "cast_info.csv", schema = schema_cast_info).persist()

cast_info.createOrReplaceTempView("Cast_Info")
#cast_info.printSchema()

display(cast_info)

,id,person_id,movie_id,person_role_id,note,nr_order,role_id
0,16255404,2202581,434,2671901.0,None,9.0,2
1,31519800,3558778,434,NaN,None,NaN,9
2,16668206,2256708,434,2264435.0,None,2.0,2
3,1148671,159670,434,271127.0,None,1.0,1
4,18308464,2464312,434,2922782.0,None,NaN,2
...,...,...,...,...,...,...,...
95,32328582,3641142,3342,NaN,(audience coordinator),NaN,10
96,22749468,2939379,3342,NaN,(associate producer),NaN,3
97,32525393,3660322,3342,NaN,(fashion stylist),NaN,10
98,23245345,2985931,3342,NaN,(producer),NaN,3


#### Role_type

In [ ]:
#=============
# Role_Type
#=============
schema_role_type = "id INT, role STRING"

role_type = spark.read.csv(path = dir + "role_type.csv", schema = schema_role_type).persist()

role_type.createOrReplaceTempView("Role_Type")
#role_type.printSchema()

display(role_type)

,id,role
0,1,actor
1,2,actress
2,3,producer
3,4,writer
4,5,cinematographer
5,6,composer
6,7,costume designer
7,8,director
8,9,editor
9,10,miscellaneous crew


#### Name
La table Name affecte un identifiant id à une personne

In [ ]:
#==============
# Name 
#==============
schema_name = """
  id int,
  name string,
  imdb_index string,
  imdb_id int,
  gender string,
  name_pcode_cf string,
  name_pcode_nf string,
  surname_pcode string,
  md5sum string
"""

name = spark.read.csv(path = dir + "name.csv", schema = schema_name).persist()

name.createOrReplaceTempView("Name")
#name.printSchema()

display(name)

,id,name,imdb_index,imdb_id,gender,name_pcode_cf,name_pcode_nf,surname_pcode,md5sum
0,333,"5, Johnny",None,NaN,m,J5,None,None,c845df6a62d46f897c7620f00f1e97a5
1,339,50 Cent,None,NaN,m,C53,None,None,faba8d4ac1b5880ef64a5de3ea7d1fa9
2,400,"A El Rahim, Ahmed",None,NaN,m,A4653,A5346,A465,9698c31ec0eb8b38172ae89e8c67b028
3,457,"A., Kodanda Rami Reddy",None,NaN,m,A2353,K3536,A,64223f8728574d3dce39126c365b5dc9
4,471,"A., Randy",None,NaN,m,A653,R53,A,b5301fd59f93e50ab8893e9b80e3a35a
...,...,...,...,...,...,...,...,...,...
95,7766,"Adamczak, Alfred",None,NaN,m,A3524,A4163,A352,9f810e511b9abfccaae85efedec24dec
96,7822,"Adamec, Jirí",None,NaN,m,A3526,J6352,A352,98cb8be3aa5048fab78c855c6a16abb8
97,7825,"Adamec, Lukás",None,NaN,m,A3524,L2352,A352,5095421885465f116cf77777b010ebcc
98,7915,"Adamo, Antonio",None,NaN,m,A3535,A5353,A35,703b6dd48dad5980b9d12e6b79bbb989


#### Person_Info

In [ ]:
#==============
# Person_Info
#==============
schema_person_info = """
  id int,
  person_id int,
  info_type_id int,
  info string,
  note string
"""

person_info = spark.read.csv(path = dir + "person_info.csv", schema = schema_person_info).persist()
person_info.createOrReplaceTempView("Person_Info")

display(person_info)

,id,person_id,info_type_id,info,note
0,302,333,33,(2012) Portlandia,None
1,303,333,26,"Martini-Connally, John",None
2,304,333,20,"Los Angeles, California, USA",None
3,305,333,17,Has worked & studied alongside studio engineer...,None
4,306,333,17,Bachelors in Digital Media Production from the...,None
...,...,...,...,...,...
95,397,339,36,"""Entertainment Weekly"" (USA), 23 February 2003...",None
96,398,339,26,"III, Curtis James Jackson",None
97,399,339,17,Is involved in a feud with fellow rapper 'Ja R...,None
98,400,339,17,Is signed to 'Eminem' (qv)'s Shady Records imp...,None


#### Movie_Companies

In [ ]:
#==============
# Movie_Companies
#==============
schema_movie_companies = """
    id int,
    movie_id int,
    company_id int,
    company_type_id int,
    note string
"""

movie_companies = spark.read.csv(path = dir + "movie_companies.csv", schema = schema_movie_companies).persist()

movie_companies.createOrReplaceTempView("Movie_Companies")
#movie_companies.printSchema()

display(movie_companies)

,id,movie_id,company_id,company_type_id,note
0,1318,3342,216,1,(2000) (USA) (TV)
1,1319,3342,217,1,(200?) (Japan) (TV)
2,1454,3981,46,1,(2009) (Hungary) (TV)
3,2889,8413,423,1,(2007) (Japan) (DVD)
4,2890,8413,49,1,(1987-1990) (USA) (TV) (original airing)
...,...,...,...,...,...
95,1297300,85610,789,2,None
96,1297301,85610,72715,2,None
97,1297403,85745,21107,2,None
98,1297588,86268,72735,2,None


#### Company_Name

In [ ]:
schema_company_name = """
    id int,
    name string,
    country_code string,
    imdb_id int,
    name_pcode_nf string,
    name_pcode_sf string,
    md5sum string
"""

company_name = spark.read.csv(path = dir + "company_name.csv", schema = schema_company_name)
company_name.createOrReplaceTempView("Company_Name")
#company_name.printSchema()

display(company_name)

,id,name,country_code,imdb_id,name_pcode_nf,name_pcode_sf,md5sum
0,1,E! Entertainment Television,[us],NaN,E5363,E5363,ddc1ca3453a33526ad931106cb60d004
1,2,YouTube,[us],NaN,Y31,Y312,3bd77ab33669d8883d345ee754928caf
2,4,5*,[gb],NaN,None,G1,2c0b57f4db6ba181ca8e7414284e9d35
3,5,Canadian Television (CTV),[ca],NaN,C5353,C5353,7bde1000f06906373c3ae6a48d981511
4,6,Columbia Broadcasting System (CBS),[us],NaN,C4516,C4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
...,...,...,...,...,...,...,...
95,129,KIKA,[de],NaN,K2,K23,561629423916c2a71f4aa5863f9388ec
96,132,Comcast,[us],NaN,C523,C5232,65403fe5213a23ff74526488eee7b1d1
97,133,NHK,[jp],NaN,N2,N21,b43377605ac37f1e62dd047e04f1c135
98,136,Turner Broadcasting System (TBS),[us],NaN,T6561,T6561,c6bcb939ad73a310ad2c3e529938eb15


#### Company_type

In [ ]:
#==============
# Company_Type
#==============
schema_company_type = "id INT, kind STRING"

company_type = spark.read.csv(path = dir + "company_type.csv", schema = schema_company_type).persist()
company_type.createOrReplaceTempView("Company_Type")
#company_type.printSchema()

display(company_type)

,id,kind
0,1,distributors
1,2,production companies
2,3,special effects companies
3,4,miscellaneous companies


### Detail sur les personnes et les oeuvres

#### TitleDetail

In [ ]:
%%sql
create or replace temp view TitleDetail as
select t.id, t.title, t.production_year, k.kind, it.info as property, m.info as value
from Title t, Movie_Info m, Info_type it, Kind_Type k
where t.id = m.movie_id 
and m.info_type_id = it.id
and t.kind_id = k.id
;

select * 
from TitleDetail t
order by  t.production_year desc, t.title
limit 100;

,id,title,production_year,kind,property,value
0,2310404,Star Trek 3,2016,movie,genres,Action
1,2310404,Star Trek 3,2016,movie,color info,Color
2,2310404,Star Trek 3,2016,movie,release dates,USA:2016
3,2310404,Star Trek 3,2016,movie,genres,Sci-Fi
4,2310404,Star Trek 3,2016,movie,genres,Adventure
...,...,...,...,...,...,...
95,1901169,Frank,2014,movie,plot,"Staggered by the death of his father, Frank, a..."
96,1901169,Frank,2014,movie,locations,"New York City, New York, USA"
97,1901169,Frank,2014,movie,locations,"Marbella, Málaga, Andalucía, Spain"
98,1901169,Frank,2014,movie,budget,"$18,000,000"


Mémoriser le résultat de TitleDetail pour éviter de le calculer plusieurs fois

In [ ]:
%%sql
cache table TitleDetail

""


#### NameDetail

In [ ]:
%%sql
create or replace temp view NameDetail as
select n.id, n.name, n.gender, i.info as property, p.info as value
from Name n, Person_Info p, Info_type i
where n.id = p.person_id
and p.info_type_id = i.id
;
 
select * 
from NameDetail n
where n.name like 'Smi%'
order by n.name, n.property
limit 100;

,id,name,gender,property,value
0,2544489,"Smid, Coosje",f,trivia,Daughter of 'Ernst Daniël Smid' (qv) and Rosem...
1,1454381,"Smid, Ernst Daniël",m,birth date,6 May 1953
2,1454381,"Smid, Ernst Daniël",m,birth notes,"Enschede, Overijssel, Netherlands"
3,1454381,"Smid, Ernst Daniël",m,magazine cover photo,"""Ncrv Gids"" (Netherlands), 17 April 1999, Iss. 16"
4,1454381,"Smid, Ernst Daniël",m,other works,Provided the voice of Gouverneur Ratcliffe in ...
...,...,...,...,...,...
95,1454554,"Smiley, Tavis",m,magazine cover photo,"""Black Enterprise"" (USA), March 2002, Vol. 32,..."
96,1454554,"Smiley, Tavis",m,trivia,Distant cousin of 'T.D. Jakes' (qv) and 'James...
97,1454554,"Smiley, Tavis",m,trivia,"Is a member of Kappa Alpha Psi Fraternity, Inc..."
98,1454554,"Smiley, Tavis",m,where now,"(2006) Release of his book, ""What I Know for S..."


Mémoriser le résultat de NameDetail pour éviter de le calculer plusieurs fois

In [ ]:
%%sql
cache table NameDetail;

""


## QUESTIONS

### Exercice 1: Extraction d'information

Définir ces relations apportant des informations structurées sur les personnes. Le schéma de chaque table est donné à titre indicatif et vous pouvez l'adapter et le compléter si nécessaire.

#### Question 1 : Salary
 **Salary** (id, name, titleId, titleName, titleYear, salary, currency, frequency, extra) avec
  - *id* et *name* sont l'id et le nom de la personne
  - *titleId*, *titleName* et *titleYear* sont l'id, le titre et l'année du film
  - *salary* est le salaire perçu par une personne pour un film
  - *currency* est la devise du salaire
  - *frequency* indique si le salaire est pour le film, un épisode de la série, une semaine de tournage, etc...
  - *extra* est un complément de salaire (exple pourcentage sur le revenu du film)
 

Voir la propriété *salary history*. Le champ *titleId* peut être nul si aucune correspondance n'est trouvée dans la table TitleDetail. Le cas échéant, essayer de trouver une correspondance avec le fichier *title_all* contenant tous les films.

Remarque: Vous pouvez (mais ce n'est pas obligatoire) utiliser les associations entre une personne et un film contenues dans Cast_Info. Cependant il faudra compléter votre solution pour les références aux films qui ne seraient pas incluses dans Cast_info. 

## Inspection des valeurs possibles en filtrant la propriété salary history

In [ ]:

%%sql 
select * from NameDetail n where n.property="salary history"

,id,name,gender,property,value
0,1989,"Abbott, Bud",m,salary history,"_Rio Rita (1942)_ (qv)::$75,000"
1,1989,"Abbott, Bud",m,salary history,"_One Night in the Tropics (1940)_ (qv)::$17,500"
2,1989,"Abbott, Bud",m,salary history,"_""The Abbott and Costello Show"" (1952)_ (qv)::..."
3,1989,"Abbott, Bud",m,salary history,"_In the Navy (1941)_ (qv)::$25,000 + 5% of pro..."
4,1989,"Abbott, Bud",m,salary history,"_Buck Privates (1941)_ (qv)::$25,000 + 5% of p..."
...,...,...,...,...,...
95,222983,"Cage, Nicolas",m,salary history,_Ghost Rider: Spirit of Vengeance (2011)_ (qv)...
96,223566,"Cain, Dean",m,salary history,"_""Lois & Clark: The New Adventures of Superman..."
97,223566,"Cain, Dean",m,salary history,"_""Lois & Clark: The New Adventures of Superman..."
98,223566,"Cain, Dean",m,salary history,"_""Lois & Clark: The New Adventures of Superman..."


### Remarques:
* colone "value" correspondant a salary history est complexe et contient plusieurs informations, elle doit être parsée afin d'en extraire les sous-informations importantes pour générer la table Salary (valeur du salaire, devise, fréquence, extra )
* format obsérvé: _titleName(titleYear)_(qv)::currencysalary/frequency + extra% of profits
* pour certaines value, le salaire précède la currency
* / parfois remplacé par "per"
* "+" parfois remplacé par "plus"
* la valeur de certains extra n'est pas spécifié, information donnée: % of gross ou percentage of gross
* il y a certains titres qui sont entre "" et d'autres pas 
* certains salaires contiennent des mots/lettres tel que 'm', 'million, 'M', 'Million'....
* Le nom et l'année du film sont aussi spécifiés dans value, la seule information manquante concernant les films est TitleId. Nous pouvons la récupérer grâce à une jointure leftjoin avec la table en cache "TitleDetail"
* Il y a des champs "value" ou la fréquence pour le salaire n'est pas spécifiée. En premier lieu, je fais l'hypothèse que quand la fréquence n'est pas spécifiée, alors elle correspond au salaire pour tout le film

Dans ce qui suit, vous trouverez les différentes fonctions qui permettent de parser value et récuperer les valeurs voulues en fonctions des caractéristiques et patterns observés et cités ci-dessus

In [ ]:
%%sql

create or replace temp view Salary_temp as (
select id, name,
      TRIM(regexp_extract(value, '_(.+)\\((.+)_', 1)) as title,
       regexp_extract(value, '_(.+)\\((.+)\\)_', 2) as year, 
       regexp_extract(split(value, ('::'))[1], '([0-9,millionMillion. ]+)')as salary,
       if(split(value, ('::'))[1] like '% per %', 
          regexp_extract(split(value, ('::'))[1], '(.*) per (.+)', 2),
          regexp_extract(split(value, ('::'))[1], '(.*)/(.+)', 2)) as frequency,
       if(split(value, ('::'))[1] like '% plus %', 
          regexp_extract(split(value, ('::'))[1], '(plus.*)'),
          regexp_extract(split(value, ('::'))[1], '(\\+.*)')) as extra,
      if (regexp_extract(split(value, '::')[1], '([^0-9 ]*)([0-9,millionMillion. ]+)', 1)=='',
          regexp_extract(split(value, '::')[1], '([^0-9 ]*)([0-9,millionMillion. ]+) ([^ >,+%\\(-]+)', 3),
        regexp_extract(split(value, '::')[1], '([^0-9 ]*)([0-9,millionMillion. ]+)', 1)) as currency
       
from NameDetail where property = "salary history" );

""


### Récupération de titleId grâce à une jointure 
jointure avec left join afin que les titres non présents dans la table title soient mis à null

In [ ]:
%%sql
create or replace temp view salary_union as (
  select s.id, s.name, t.id as titleId, t.title as TitleName, 
        s.salary, s.frequency, s.extra, s.currency
  from Salary_temp s
  left join TitleDetail t
  where TRIM(s.title)=TRIM(t.title) and s.year=t.production_year
  UNION
  select s.id, s.name, t_a.id as titleId, t_a.title as titleName, 
        s.salary, s.frequency, s.extra, s.currency 
  from Salary_temp s
  left join title_all t_a
  where TRIM(s.title)=TRIM(t_a.title) and s.year=t_a.production_year
);

create or replace temp view Salary as (
  select distinct id, TRIM(name) as name, titleId, TRIM(titleName) as titleName,
          salary, frequency, extra, currency
          from salary_union
);

select * from Salary

,id,name,titleId,titleName,salary,frequency,extra,currency
0,793040,"Keaton, Buster",1736753,Blue Blazes,2500,,,$
1,434515,"Eastwood, Clint",1901066,Francis in the Navy,100,,,$
2,1939686,"De France, Cécile",1085025,Hereafter,"300,000",,,EUR
3,1158978,"Olivier, Laurence",2381490,The Jazz Singer,"1,000,000",,,$
4,158067,"Bogart, Humphrey",2439160,To Have and Have Not,2750,week,,$
...,...,...,...,...,...,...,...,...
95,2182322,"Kidman, Nicole",1730451,Birth,"15,000,000",,,$
96,471029,"Farley, Chris",1169943,Black Sheep,"6,000,000",,,$
97,471029,"Farley, Chris",1733015,Black Sheep,"6,000,000",,,$
98,1318653,"Robinson, Edward G.",1733448,Blackmail,"8,500",week,,$


#### Question 2 : Height
 **Height**(id, name, heightInCm)  recense la taille d'une personne de manière exploitable, en les reconvertissant en cm, alors que les entrées de la base ont différentes unités de mesure.

## Inspection des valeurs possibles en filtrant la propriété height dans NameDetail

In [ ]:
%%sql 
select * from NameDetail n where n.property="height" 

,id,name,gender,property,value
0,339,50 Cent,m,height,6'
1,750,"Aames, Willie",m,height,"5' 6"""
2,1521,"Abatantuono, Diego",m,height,183 cm
3,1989,"Abbott, Bud",m,height,"5' 8"""
4,2133,"Abbott, Philip",m,height,"5' 10"""
...,...,...,...,...,...
95,33751,"Altobello III, John",m,height,"5' 9"""
96,34692,"Alvarez, Ronnie",m,height,"5' 7"""
97,35305,"Alzado, Lyle",m,height,"6' 3"""
98,35903,"Amar, Matt",m,height,"5' 8"""


###Remarques:
En explorant les données, je vois qu'il y a deux types d'unités de mesures
* cm
* ' " ( ':foot, ": inch , standard US )

Je fais un test pour voir si ce sont les uniques formats présents:

In [ ]:
%%sql
with T1 as (
select id,  
      if(n.value LIKE '%cm%', n.value, if(n.value LIKE "%\'%", 'b', 'c' )) as v
from NameDetail n
where n.property="height" )

select t.id, t.v , n.value
from T1 t, NameDetail n
where t.v = 'c' and n.id=t.id and n.property="height";

,id,v,value
0,487104,c,181


### Remarque:
* La requete T1 permet de retourner la valeur en cm si elle existe, 'b' si la valeur est sous le format ' " et 'c' sinon
* La présence d'un tuple retourné avec la condition de la valeur ='c' signifie qu'il existe bien un autre format. Celui-ci correspond à la présence d'une valeur sans qu'une unité de mesure ne soit spécifiée

In [ ]:
%%sql
select id, n.value
from NameDetail n
where property="height" and id=295225

,id,value
0,295225,168cm cm


### Remarque:
* Le format du tuple ci-dessus n'est pas conforme car cm est présent en double dans value
* Il existe un autre tuple avec le format x'z cm ( les deux unités de mesures sont mélangées )

### Tentative de contouner le problème
* hypothèse 1 : si aucune des deux unités n'est spécifié, qu'il y a plus de probabilité que ce soit cm qui soit oublié par rapport à l'oubli de '" qui coupent la valeur et qui serait donc selon moi une erreur moins probable
* hypothèse 2: s'il y a au moins une occurence de la chaine 'cm', l'unité est 'cm'
* hypothèse 3: s'il y a une occurence de la chaine "'" et pas d'occurence de la chaine "cm", alors l'unité de mesure est l'unité feet, inch...
### normalisation des données

In [ ]:
%%sql
create or replace temp view feet_inch_plus as
select id, name, 
        split(value, '\'')[0] as feet, 
        split(split(value, ' ')[1], '\"')[0]  as inch, 
        split(split(value, ' ')[2], "\"")[0] as plus
from NameDetail 
where property="height" and value like "%\'%" and value not like "%cm%";

--  vue qui sépare les feet, les inch et la fraction en plus 
create or replace temp view feet_inch as
select id, name, 
      feet, 
      if(inch is null, 0, inch) as inch, 
      if(plus is null, 0, split(plus, "/")[0]/split(plus, "/")[1]) as plus 
from feet_inch_plus;

""


In [ ]:
%%sql
-- vue qui à ne garde que les chiffres de value qui comporte "cm"
create or replace temp view cm as 
select n.id, n.name, regexp_replace(n.value, "([^0-9.])", '') as height
from NameDetail n
where n.property="height" and n.value like "%cm%"

""


In [ ]:
#Requête qui permet d'unir les deux vues e transformant les tailles en feet vers cm 

%%sql
create or replace temp view Height as 
select id, name, 
      concat(height, " cm") as heightInCm
from cm
UNION
select id, name, 
      concat(round((feet*12 + (inch+plus))*2.54, 2), " cm") as heightInCm 
from feet_inch;

select * from Height

,id,name,heightInCm
0,1521,"Abatantuono, Diego",183 cm
1,8537,"Adams, Isiah",182 cm
2,43482,"Andina, Fernando",183 cm
3,45910,"Andrés López, Ángel de",170 cm
4,47771,"Ankermann, Martin",184 cm
...,...,...,...
95,667832,"Herold, Volker",179 cm
96,681840,"Hodge, Aldis",186 cm
97,682788,"Hoening, Christian",181 cm
98,688399,"Holmes, Eamonn",182 cm


#### Question 3 : Magazine

 **MagazineCover**(id, name, magazine, country, day, month, year) donnant le nom du magazine dans lequel la personne est sur la couverture, avec le pays du magazine et la date de parution. Voir la propriété *magazine cover photo*.

### Observation du format

In [ ]:

%%sql 
select * from NameDetail n where n.property="magazine cover photo"

,id,name,gender,property,value
0,339,50 Cent,m,magazine cover photo,"""Big Fish"" (Greece), 4 June 2006, Iss. 28"
1,339,50 Cent,m,magazine cover photo,"""Hitkrant"" (Netherlands), 19 March 2005, Iss. 11"
2,339,50 Cent,m,magazine cover photo,"""The Source"" (USA), October 2003, Iss. 169"
3,339,50 Cent,m,magazine cover photo,"""Breakout!"" (Netherlands), 7 August 2003, Iss. 33"
4,339,50 Cent,m,magazine cover photo,"""Dub"" (USA), 2003, Iss. 15"
...,...,...,...,...,...
95,64035,"Ashcroft, Richard",m,magazine cover photo,"""Melody Maker"" (UK), 3 January 1998"
96,64035,"Ashcroft, Richard",m,magazine cover photo,"""Melody Maker"" (UK), 11 October 1997"
97,64035,"Ashcroft, Richard",m,magazine cover photo,"""NME"" (UK), 7 June 1997"
98,67883,"Atkinson, Rowan",m,magazine cover photo,"""Preview"" (Netherlands), 1 April 2007, Iss. 2"


### Remarque:
* format observé dans value: "magazine" (country), day month year, vol. number, Iss. number
* Je remarque parmis un échantillon de valeurs observé qu'il y a des champs value pour lesquels les champs "vol. number" ou "Iss. number" ne sont pas spécifiés ( ou inclusif )
* Certains champs value ne contiennent pas day ou month ( ou inclusif )
* Day: peut avoir un ou deux caractères de type chiffre, à ne pas confondre avec un nom de magazine qui a le même format. Exemple: "21 magazine"(Iran)
* Pour certains champs country, plus de détails sont spécifiés. Exemple: "West Germany". En regardant les dates, nous remarquons que la date associé à ce type de cas correspond à l'époque ou l'Allemagne était séparée en deux. Nous considérons donc que garder l'information tel quel reste pertinent
* Sur certaines champs value, il y a plusieurs valeurs entre parenthèses qui se suivent. Nous considérons que c'est la parenthèse présente après la première virgule qui spécifie le pays
* year au format: YYYY avec Y \in [0, 9]. Specifier la condition d'apparition avant Iss et Vol pour ne pas confondre avec les chiffres suivant ces derniers 


In [ ]:
%%sql
create or replace temp view MagazineCover as 
select id, name, value,
        TRIM(regexp_extract(value,'\\"(.+)\\"', 1)) as magazine, 
        TRIM(regexp_extract(split(value,',')[0], "^.*\\((.+)\\).*$")) as country,
        regexp_extract(value, '[^,]+, (Vol\\. [0-9]+, )?(Iss\\. [0-9]+, )?([0-9]{1,2}) ', 3) as day,
        regexp_extract(value,'[^,]+, (Vol\\. [0-9]+, )?(Iss\\. [0-9]+, )?[^a-zA-Z]*([a-zA-Z\\-/ ]*)( [0-9]|$|,)', 3) as month,
        regexp_extract(value, '[^,]+, (Vol\\. [0-9]+, )?(Iss\\. [0-9]+, )?[^a-zA-Z]{0,2}[a-zA-Z \\-/]*[^a-zA-Z]{0,2}(, )?([0-9]{4})', 4) as year
from NameDetail 
where property = "magazine cover photo" ;

select * from MagazineCover

,id,name,value,magazine,country,day,month,year
0,339,50 Cent,"""Big Fish"" (Greece), 4 June 2006, Iss. 28",Big Fish,Greece,4,June,2006
1,339,50 Cent,"""Hitkrant"" (Netherlands), 19 March 2005, Iss. 11",Hitkrant,Netherlands,19,March,2005
2,339,50 Cent,"""The Source"" (USA), October 2003, Iss. 169",The Source,USA,,October,2003
3,339,50 Cent,"""Breakout!"" (Netherlands), 7 August 2003, Iss. 33",Breakout!,Netherlands,7,August,2003
4,339,50 Cent,"""Dub"" (USA), 2003, Iss. 15",Dub,USA,,,2003
...,...,...,...,...,...,...,...,...
95,64035,"Ashcroft, Richard","""Melody Maker"" (UK), 3 January 1998",Melody Maker,UK,3,January,1998
96,64035,"Ashcroft, Richard","""Melody Maker"" (UK), 11 October 1997",Melody Maker,UK,11,October,1997
97,64035,"Ashcroft, Richard","""NME"" (UK), 7 June 1997",NME,UK,7,June,1997
98,67883,"Atkinson, Rowan","""Preview"" (Netherlands), 1 April 2007, Iss. 2",Preview,Netherlands,1,April,2007


In [ ]:
%%sql
select * from MagazineCover m where m.month="Janauary"

,id,name,value,magazine,country,day,month,year
0,2050551,"Gish, Annabeth","""TV Zone"" (UK), Janauary 2002, Vol. 147",TV Zone,UK,,Janauary,2002


In [ ]:
%%sql
select * from MagazineCover m where m.month="Winter"

,id,name,value,magazine,country,day,month,year
0,770566,"Jurasik, Peter","""Universe Today"" (USA), Winter 1998, Vol. 6, I...",Universe Today,USA,,Winter,1998
1,1808723,"Bennett, Joan","""Scarlet Street"" (USA), Winter 1993, Vol. 9",Scarlet Street,USA,,Winter,1993
2,1836847,"Boyle, Lara Flynn","""The World of Hibernia"" (USA), Winter 1999, Vo...",The World of Hibernia,USA,,Winter,1999
3,2030581,"Gabrielle, Monique","""Scream Queens Illustrated "" (USA), Winter 199...",Scream Queens Illustrated,USA,,Winter,1994
4,2124884,"Hudson, Kate","""Teen Tribute"" (Canada), Winter 2000, Vol. 03,...",Teen Tribute,Canada,,Winter,2000
5,2127689,"Hunter, Holly","""Filmmaker"" (USA), Winter 1997",Filmmaker,USA,,Winter,1997
6,2320861,"Milano, Alyssa","""Sirens of Cinema"" (USA), Winter 2001, Iss. 1",Sirens of Cinema,USA,,Winter,2001
7,2392675,"Packard, Kelly","""Diet & Exercise Magazine"" (USA), Winter 1997",Diet & Exercise Magazine,USA,,Winter,1997
8,2417452,"Pfeiffer, Michelle","""Femme Fatales"" (USA), Winter 1992, Vol. 1, Is...",Femme Fatales,USA,,Winter,1992
9,2530388,"Shields, Brooke","""24 Hour Fitness"" (USA), Winter 1997",24 Hour Fitness,USA,,Winter,1997


In [ ]:
%%sql
select * from MagazineCover m where m.month="Summer"

,id,name,value,magazine,country,day,month,year
0,174834,"Boxleitner, Bruce","""Universe Today"" (USA), Summer 1997, Vol. 5, I...",Universe Today,USA,,Summer,1997
1,229844,"Campbell, Glen","""Midwest Today"" (USA), Summer 1994, Iss. 9",Midwest Today,USA,,Summer,1994
2,347381,"Dalton, Timothy","""Preview"" (Netherlands), Summer 1989",Preview,Netherlands,,Summer,1989
3,411281,"Dorough, Howie","""Teen People"" (USA), Summer 2000",Teen People,USA,,Summer,2000
4,1021038,"McLean, A.J.","""Teen People"" (USA), Summer 2000",Teen People,USA,,Summer,2000
5,1114181,"Navarro, Dave","""Smoke"" (USA), Summer 1996",Smoke,USA,,Summer,1996
6,1473414,"Spacey, Kevin","""Smoke"" (USA), Summer 1998, Vol. 3, Iss. 3",Smoke,USA,,Summer,1998
7,1921940,"Cruz, Penélope","""Detour"" (USA), Summer 2000",Detour,USA,,Summer,2000
8,2187323,"Kirkland, Sally","""Femme Fatales"" (USA), Summer 1994, Vol. 3, Is...",Femme Fatales,USA,,Summer,1994
9,2252009,"Lopez, Jennifer","""Viasat Magazine"" (Sweden), Summer 2000, Iss. 6",Viasat Magazine,Sweden,,Summer,2000


In [ ]:
%%sql
select * from MagazineCover m where m.month="Spring"

,id,name,value,magazine,country,day,month,year
0,95899,"Barks, Carl","""SWOF"" (Switzerland), Spring 1999 , Iss. 27 (US)",SWOF,Switzerland,,Spring,1999
1,191540,"Brokaw, Tom","""Midwest Today"" (USA), Spring 1995, Iss. 12",Midwest Today,USA,,Spring,1995
2,280046,"Christensen, Hayden","""Men's Fashions of the Times"" (USA), Spring 2002",Men's Fashions of the Times,USA,,Spring,2002
3,280046,"Christensen, Hayden","""Teen Vogue"" (USA), Spring 2002",Teen Vogue,USA,,Spring,2002
4,330271,"Cronkite, Walter","""Midwest Today"" (USA), Spring 1997, Iss. 20",Midwest Today,USA,,Spring,1997
5,372266,"Dean, James","""Campus USA"" (USA), Spring 1988",Campus USA,USA,,Spring,1988
6,729881,"Jackson, Andrew","""Parsec"" (Canada), Spring 2000, Vol. 3, Iss. 5",Parsec,Canada,,Spring,2000
7,883819,"Lea, Nicholas","""The X-Files Official Magazine"" (USA), Spring ...",The X-Files Official Magazine,USA,,Spring,1998
8,1672219,"White, Mike","""Filmmaker"" (USA), Spring 2000",Filmmaker,USA,,Spring,2000
9,1755487,"Alley, Kirstie","""Midwest Today"" (USA), Spring 1994, Iss. 8",Midwest Today,USA,,Spring,1994


In [ ]:
%%sql
select distinct regexp_extract(value,'[^,]+, (Vol\\. [0-9]+, )?(Iss\\. [0-9]+, )?[^a-zA-Z]*([a-zA-Z\\-/ ]*)( [0-9]|$|,)', 3) as month
from NameDetail 
where property = "magazine cover photo" 
LIMIT 8

,month
0,March
1,
2,July
3,January
4,May
5,July-August
6,November/December
7,June-July


In [ ]:
%%sql
select * from MagazineCover m where m.month="Dallas Times Herald"

,id,name,value,magazine,country,day,month,year
0,121266,"Beltran, Robert","""TV Times"" (USA), Dallas Times Herald, 20 May ...",TV Times,USA,20,Dallas Times Herald,1984


###Remarque: valeurs mal formatées
* Certains champs extraits dans Month sont mal formatés. Dans la cellule ci-dessus, nous remarquons une faute de syntaxe ( Janauary au lieu de January )
* Certaines champs dans Month correspondent à des saisons. Nous pourrions considérer de rajouter une colonne pour les saisons
* Nous pourrions remédier à cela en faisant des correspondances entre les valeurs abérantes avec l'intervalle de 12 mois bien formaté
* Certains champs month contiennent plusieurs mois séparés par des "/" ou des "-"
* Pour la valeur abérante: Dallas Times Herald, nous remarquons qu'il y a un nom de magazine en plus qui n'est respecte pas le format général ( qui n'est pas entre ""). 
***Conclusion*** 
Il faudrait refaire une deuxième passe sur le tableau résultat pour gérer ces cas particuliers


#### Question 4 : Trivia
Cette question concerne l'extraction de références à des personnes ou à des films dans une description textuelle. L'objectif est de proposer une méthode pour retrouver des références dans un contenu où les références ne sont pas toujours repérées explicitement.
On considère la propriété *trivia* dans NameDetail : on constate que le contenu textuel peut contenir des références à des films ou à d'autres personnes.

Définir deux tables détaillant les anecdotes connues sur des personnes :
- **TitleTrivia** (fromPersonId, fromPersonName, toTitleId, toTitleName, triviaText)
- **NameTrivia** (fromPersonId,  fromPersonName, toPersonId, toPersonName, triviaText)
    
Indications :  
Les attributs *fromPersonId* et *fromPersonName* sont l'id et le nom d'une personne dans NameDetail.

Pour l'attribut *toTitleId* dans TitleTrivia : si le film n'est pas référencé dans la table TitleDetail, essayer de trouver une correspondance dans le fichier contenant tous les films (*cf.* title_full).

Pour le champ *toPersonId* dans NameTrivia : si la personne n'est pas référencée dans la table NameDetail, essayer de trouver une correspondance dans le fichier contenant toutes les personnes (*cf.*  name_full).

Remarque : dans cette question, on se limite aux anecdotes liées à des personnes (propriété *trivia* dans NameDetail). On ne considère pas les anecdoctes liées à des films (propriété *trivia* dans TitleDetail).

In [ ]:
%%sql 
select * from NameDetail n where n.property="trivia"

,id,name,gender,property,value
0,333,"5, Johnny",m,trivia,Has worked & studied alongside studio engineer...
1,333,"5, Johnny",m,trivia,Bachelors in Digital Media Production from the...
2,333,"5, Johnny",m,trivia,Nephew of writer/director 'Richard Martini (I)...
3,333,"5, Johnny",m,trivia,Related to former Texas Governor & U.S. Secret...
4,339,50 Cent,m,trivia,Is involved in a feud with fellow rapper 'Ja R...
...,...,...,...,...,...
95,3892,"Abineri, Daniel",m,trivia,"Has one daughter, Lola (b. 1998)"
96,3892,"Abineri, Daniel",m,trivia,"Married to Claudia Rosencrantz, controller of ..."
97,3963,"Abkarian, Simon",m,trivia,Armenian like the genius 'Sergei Parajanov' (qv).
98,3963,"Abkarian, Simon",m,trivia,Taught at CNSAD (Conservatoire National Superi...


Nous remarquons que le champs value n'a pas de structure précise. Certaines descriptions contiennent plusieurs propriétés au même temps. Certaines commencent par des verbes, d'autres par des noms et d'autres par un pronom ... Donc spécifier le type de relation est dur. Cependant, nous pouvons éudier la structure des phrases pour récupérer les titres de films ou les noms de personnes.

In [ ]:
%%sql 
select * from NameDetail n where n.property="trivia" and value like '%(qv)%'

,id,name,gender,property,value
0,333,"5, Johnny",m,trivia,Has worked & studied alongside studio engineer...
1,333,"5, Johnny",m,trivia,Nephew of writer/director 'Richard Martini (I)...
2,333,"5, Johnny",m,trivia,Related to former Texas Governor & U.S. Secret...
3,339,50 Cent,m,trivia,Is involved in a feud with fellow rapper 'Ja R...
4,339,50 Cent,m,trivia,Is signed to 'Eminem' (qv)'s Shady Records imp...
...,...,...,...,...,...
95,13218,"Agosto, Ben",m,trivia,2006 Olympic figure skating silver medalist wi...
96,13218,"Agosto, Ben",m,trivia,2004-2008 U.S. figure skating national champ. ...
97,15591,"Ahn, Philip",m,trivia,Brother of actors 'Philson Ahn' (qv) and 'Ralp...
98,15862,"Ahrle, Leif",m,trivia,Son of actor 'Elof Ahrle' (qv) and actress 'Bi...


###Title Trivia
extraction de références à des films 
* Nous remarquons que les titres des films sont suivi de (qv). Exemple:  _"Bibleman" (1996)_ (qv) 
* Pout faire la jointure avec Title Detail et title_all, nous récupérons l'année de production

In [ ]:
%%sql

--On récupère l'id et le nom de la personne ainsi que le film et l'année cités 
--dans l'anecdote.
create or replace temp view TitleTrivia_temp as (
  select id as fromPersonId, name as fromPersonName,  
        replace(regexp_extract(value, "_\"?(.+?)\"?\s*\\(([0-9]+)\\).*_", 1), "\"", '') as toTitleName, 
        regexp_extract(value, "_\"?(.+?)\"?\s*\\(([0-9]+)\\).*_", 2) as date,
        value as triviaText
  from NameDetail  n
  where n.property="trivia" and value like '%(qv)%');
--Normaliser le titre pour l'avoir au bon format
create or replace temp view TitleTrivia_temp_bis as 
select distinct(t.fromPersonId), t.fromPersonName, t.triviaText, if(regexp_extract(t.toTitleName, "(.+?)\"", 1)="", t.toTitleName, regexp_extract(t.toTitleName, "(.+?)\"", 1)) as toTitleName,t.date
from TitleTrivia_temp t; 
--On fait une union entre les correspondances de TitleDetail et title_all
create or replace temp view TitleTrivia_union as (
  select fromPersonId, fromPersonName, toTitleName, triviaText
  from TitleTrivia_temp_bis tt, TitleDetail nd
  where TRIM(toTitleName)=TRIM(title) and date=production_year
  UNION
  select fromPersonId, fromPersonName, toTitleName, triviaText
  from TitleTrivia_temp_bis tt, title_all ta
  where TRIM(toTitleName)=TRIM(title) and date=production_year
);

--On applique un distinct pour ne pas avoir de doublon
create or replace temp view TitleTrivia as (
  select distinct fromPersonId, 
                  TRIM(fromPersonName) as fromPersonName, 
                  TRIM(toTitleName) as toTitleName, 
                  triviaText
  from TitleTrivia_union
);

select * from TitleTrivia


,fromPersonId,fromPersonName,toTitleName,triviaText
0,368897,"De Niro, Robert",Any Given Sunday,Turned down the role of Tony D'Amato in _Any G...
1,508534,"Foxx, Jamie",Any Given Sunday,Good friends with his _Any Given Sunday (1999)...
2,932932,"Lowery, Robert",Batman and Robin,Was the second actor to play Batman (_Batman a...
3,1528273,"Talbot, Lyle",Batman and Robin,Played Commissioner Gordon in the 1949 Batman ...
4,2680857,"Wray, Fay","Broadway: The Golden Age, by the Legends Who W...","In January 2003, a 95-year-old Fay Wray was aw..."
...,...,...,...,...
95,71952,"Avery, James",The Devil's Rejects,Was considered for the role of Charlie Altamon...
96,470403,"Fargas, Antonio",The Gambler,His performance as a pimp in _The Gambler (197...
97,1889032,Cher,The Grifters,Was in consideration for the part of Lilly Dil...
98,1808062,"Bening, Annette",The Grifters,The longest she has gone without an Oscar nomi...


###Name Trivia
extraction de références à des personnes
* les noms de personnes se trouvent avant (qv). Exemple: 'Adrien Brody' (qv) and 'Robert Evans (I)' (qv) won awards 
* À niter que certains Name ne sont pas pris en compte car ils ne se trouvent pas avant (qv) mais après un with. Exemple: in _Eddie (1996)_ (qv) with 'Whoopi Goldberg'. Aussi que lors de la présence de plusieurs noms dans le détails, tous ne sont pas détéctés

In [ ]:
%%sql

create or replace temp view T4 as 
select n1.id as fromPersonId, n1.name as fromPersonName, regexp_extract(n1.value,".*'(.+?)' [(]qv[)]",1) as toPersonName, n1.value as triviaText
from NameDetail  n1
where n1.property="trivia" and regexp_extract(n1.value,".*'(.+?)' [(]qv[)]",1)!="";

create or replace temp view NameTrivia as 
select distinct(t.fromPersonId), t.fromPersonName, n.id as toPersonId, t.toPersonName, t.triviaText
from T4 t, name_all n
where concat(n.name,imdb_index)=concat(split(t.toPersonName," ")[1],", ",split(t.toPersonName," ")[0], regexp_extract(t.toPersonName,"[(](.+?)[)]",1) ) ;

select * from NameTrivia;

,fromPersonId,fromPersonName,toPersonId,toPersonName,triviaText
0,816627,"Kirby, Bruno",2748,Kareem Abdul-Jabbar (I),Went to high school and was friends with 'Kare...
1,963846,"Malone, Karl",2748,Kareem Abdul-Jabbar (I),(5 December 2000) Surpassed 'Wilt Chamberlain'...
2,1891611,"Chisholm, Melanie",8178,Bryan Adams (I),Dueted with 'Bryan Adams (I)' (qv) on a song c...
3,245868,"Carter, Nick",8178,Bryan Adams (I),Lists Canadian rocker 'Bryan Adams (I)' (qv) a...
4,2342498,"Morrison, Rana",8233,Chris Adams (V),"Intrigued at first with wrestling, she actuall..."
...,...,...,...,...,...
95,100841,"Barry, Gene",101123,John Barrymore (I),Changed his name to Gene Barry in honor of his...
96,1914687,"Costello, Helene",101123,John Barrymore (I),Sister of actress 'Dolores Costello' (qv). For...
97,462573,"Evans, Maurice",101123,John Barrymore (I),Although 'John Barrymore (I)' (qv) is still co...
98,682398,"Hodiak, John",1799880,Anne Baxter (I),Father of actress 'Katrina Hodiak' (qv) from h...


#### Question 5 : Lien IMDB-YAGO

On suppose que certaines personnes sont présentes à la fois dans IMDB et dans YAGO mais elles ne sont pas structurées de la même façon.

5a) Compléter les Personnes existant dans NameDetail avec des informations venant de yago (voir **yagoPerson** et **actorFacts**). 
Pour cela, définir la relation qui donne le lieu de naissance et le conjoint (ou la conjointe) d'une personne :

  - **NameYago** (id, name, yagoName, yagoBirthPlace, yagoSpouse) 

5b) Compléter le contenu de NameYago pour les personnes de IMDB existant dans *name_full*.

### NameYago
observation des types de structurations

In [ ]:
%%sql
select name from NameDetail LIMIT 100

,name
0,50 Cent
1,50 Cent
2,50 Cent
3,50 Cent
4,50 Cent
...,...
95,"Adams, Mason"
96,"Adams, Mason"
97,"Adams, Mason"
98,"Adams, Mason"


In [ ]:
%%sql
select split(subject, '/')[4] from yagoPerson LIMIT 20

,"split(subject, /, -1)[4]"
0,Heywood_L._Edwards>
1,Ramanichandran>
2,Yegor_Kiryakov>
3,Patrick_J._Adams>
4,Basilio_J._Valdes>
5,Meryem_Uzerli>
6,Marie-Louise_Loubet>
7,Alex_Lacey>
8,Michael_Hankinson>
9,Éric_Alard>


### Remarques
formats:
* Dans NameDetail: Nom, Prénom 
* Dans Yago: Prénom_Nom
Nous allons supprimer les "," et les "_" puis inverser les nom et prénom dans NameDetail pour générer les corrrespondances

In [ ]:
%%sql

-- Création d'une vue qui sépare le nom du prénom dans NameDetail
create or replace temp view Pers_ND_temp as 
  select distinct id, split(name, ',')[0] as l_name, split(name, ',')[1] as f_name
  from NameDetail
;

-- Création d'une vue qui inverse l'ordre du nom et prénom dans la chaine de 
-- caractères name de NameDetail
-- Si seul le nom ou le prénom existe, la chaine de caractère name est retournée 

create or replace temp view Pers_ND as
select distinct n.id, n.name, if (l_name is null or f_name is null, n.name, concat(f_name, ' ', l_name)) as name_normalized
from NameDetail n JOIN Pers_ND_temp t1 ON t1.id = n.id
;

-- Création d'une vue qui :
-- 1) Normalise les noms dans yagoPerson : extraction du nom et prénom.
-- 2) Associe à chaque personne (nom/prénom) les propriétés birthPlace et spouse
--    avec leur valeur, si elles existent
create or replace temp view Pers_Yago as 
select distinct regexp_extract(py.subject, '<http://yago-knowledge.org/resource/(.+)>',1) as name,
              if(py.subject like "%(%)%", 
                            replace(regexp_extract(py.subject, '<http://yago-knowledge.org/resource/(.+)(_\\(.+\\))>',1), '_', ' '),
                            replace(regexp_extract(py.subject, '<http://yago-knowledge.org/resource/(.+)>',1), '_', ' ') ) as name_normalized,
            regexp_extract(f.property, '<http://schema.org/(.+)>') as property,
            replace(regexp_extract(f.object, '<http://yago-knowledge.org/resource/(.+)>'), '_', ' ') as value
from yagoPerson py JOIN actorFacts f ON f.subject = py.subject 
where f.property = '<http://schema.org/birthPlace>' or f.property= '<http://schema.org/spouse>'
;

""


### Jointure + Schéma
sur le nom et prénom 

In [ ]:
%%sql
create or replace temp view NameYago as (
  select distinct TRIM(py.name) as yago_name,
                  TRIM(pnd.name) as nameD_name, 
                  TRIM(py.name_normalized) as name_normalized, 
                  bp.value as birthPlace, 
                  s.value as spouse
  from Pers_Yago py, Pers_ND pnd, 
        (select value, name_normalized from Pers_Yago where property="birthPlace") as bp,
        (select value, name_normalized from Pers_Yago where property="spouse") as s
  where TRIM(py.name_normalized) = TRIM(pnd.name_normalized)
        and TRIM(bp.name_normalized)=TRIM(pnd.name_normalized)
        and TRIM (s.name_normalized)=TRIM(pnd.name_normalized));

select * from NameYago 

,yago_name,nameD_name,name_normalized,birthPlace,spouse
0,Aditya_Pancholi,"Pancholi, Aditya",Aditya Pancholi,Mumbai,Zarina Wahab
1,Agnieszka_Perepeczko,"Perepeczko, Agnieszka",Agnieszka Perepeczko,Warsaw,Marek Perepeczko
2,Agustín_Lara,"Lara, Agustín",Agustín Lara,Mexico City,María Félix
3,Akira_Takarada,"Takarada, Akira",Akira Takarada,Korea under Japanese rule,Akiko Kojima
4,Alan_Curtis_(American_actor),"Curtis, Alan",Alan Curtis,Chicago,Yana (singer)
...,...,...,...,...,...
95,Gary_Cooper,"Cooper, Gary",Gary Cooper,"Helena, Montana",Veronica Cooper
96,Geena_Davis,"Davis, Geena",Geena Davis,"Wareham, Massachusetts",Renny Harlin
97,Georgette_Tissier,"Tissier, Georgette",Georgette Tissier,Boulogne-Billancourt,Jean Tissier
98,Gina_Alajar,"Alajar, Gina",Gina Alajar,Manila,Michael de Mesa


récupération d'information manquante depuis name_all

In [ ]:
%%sql
select y.yago_name, y.birthPlace, y.spouse, n.name from NameYago y, name_all n

,yago_name,birthPlace,spouse,name
0,Aditya_Pancholi,Mumbai,Zarina Wahab,"Abela, Mike"
1,Aditya_Pancholi,Mumbai,Zarina Wahab,"A., David"
2,Aditya_Pancholi,Mumbai,Zarina Wahab,"-Alverio, Esteban Rodriguez"
3,Aditya_Pancholi,Mumbai,Zarina Wahab,"Abbas, Athar"
4,Aditya_Pancholi,Mumbai,Zarina Wahab,"Aberer, Leo"
...,...,...,...,...
95,Aditya_Pancholi,Mumbai,Zarina Wahab,"Abeghraz, Rachid"
96,Aditya_Pancholi,Mumbai,Zarina Wahab,"Abbott, Percy"
97,Aditya_Pancholi,Mumbai,Zarina Wahab,"Abacan, Jose Mari"
98,Aditya_Pancholi,Mumbai,Zarina Wahab,"5, Channel"


### Exercice 2 : Analyse

On considère la table **RevenuWeekEndFilm**(id, titre, pays, jour, mois, année, revenu, écrans) dont les informations proviennent de la propriété *weekend gross* de TitleDetail
* un week-end est défini par les attributs (jour, mois, année)
* l'attribut *écrans* correspond à la quantité associée au terme *screens*



In [ ]:
%%sql 
select * from TitleDetail t
where t.property="weekend gross"

,id,title,production_year,kind,property,value
0,1640952,39 Pounds of Love,2005,movie,weekend gross,$90 (USA) (22 January 2006) (1 screen)
1,1640952,39 Pounds of Love,2005,movie,weekend gross,"$1,227 (USA) (15 January 2006) (1 screen)"
2,1640952,39 Pounds of Love,2005,movie,weekend gross,$160 (USA) (18 December 2005)
3,1640952,39 Pounds of Love,2005,movie,weekend gross,"$6,532 (USA) (11 December 2005)"
4,1640952,39 Pounds of Love,2005,movie,weekend gross,"$8,055 (USA) (4 December 2005) (1 screen)"
...,...,...,...,...,...,...
95,1916458,Ghost Dog: The Way of the Samurai,1999,movie,weekend gross,"£24,595 (UK) (1 June 2000) (9 screens)"
96,1916458,Ghost Dog: The Way of the Samurai,1999,movie,weekend gross,"£26,051 (UK) (25 May 2000) (8 screens)"
97,1916458,Ghost Dog: The Way of the Samurai,1999,movie,weekend gross,"£79,199 (UK) (18 May 2000) (33 screens)"
98,1916458,Ghost Dog: The Way of the Samurai,1999,movie,weekend gross,"£144,205 (UK) (11 May 2000) (37 screens)"


### Remarques
* format de value: CurrencyAmout (country) (day month year) (nb_screens screen)

#### Question 6 : Restructuration
L'objectif de cette question est de savoir définir une structure complexe contenant des attributs qui ne sont pas des valeurs simples mais les listes, des tuples, des listes de tuples, ...

Définir une table **Film** (id, titre, roles, paysSalles) avec 
 - *roles* est une liste de tuples (acteur, nom, rang) où rang est le numéro d'ordre du rôle (attribut *nr_order*)
 - *paysSalles* est la liste des pays où le film est sorti en salles, triée par revenu dans le pays

### Film
Dans un premier temps, je crée la vue Movie, qui permet de filtrer sur les oeuvres qui sont des films

In [ ]:
%%sql
create or replace temp view Movie as
select id, title
from TitleDetail 
where kind="movie";

""


Dans un deuxième temps, je fais une jointure avec Cast_info et NameDetail pour pouvoir récupérer les informations manquantes pour construire le tuple
* j'utilise la fonction max pour récupérer le titre associé, je sais que ce n'est pas la fonction la plus optimale en terme de temps de calcul, mais je préfère faire cela que de faire une jointure après. Je pense que c'est negligeable car les données sont parcourues dans tous les cas pour faire le collect_set

In [ ]:
%%sql
create or replace temp view Roles as
select distinct(m.id), max(m.title) as title, collect_set((c.person_id, n.name, c.nr_order)) as roles
from Cast_Info c, NameDetail n, Movie m
where c.person_id=n.id and m.id=c.movie_id
group by m.id;

select * from Roles;

,id,title,roles
0,1633601,'Til Death,"[(368956, De Ocampo, Ramon, 1), (555505, Giaca..."
1,1635519,"11,830,420","[(3326648, Bradley, Marc, None)]"
2,1636236,15.35: spoor 1,"[(376663, Dekens, Rolf, None), (1454181, Smeet..."
3,1639978,2÷3,"[(3181538, Press, Richard, None), (2758983, Ch..."
4,1640259,3 Ninjas: High Noon at Mega Mountain,"[(602534, Grove, Mark Steven, None), (843014, ..."
...,...,...,...
95,1704268,Avariya,"[(11175, Adomaitis, Regimantas, None), (771870..."
96,1704921,Awaken,"[(210417, Burke, Al, 8), (3270862, Kuo, Thomas..."
97,1705109,Awesome Guy: A New Identity,"[(613595, Guzman, Mario, None), (3590464, Bart..."
98,1705185,Awoken,"[(3044180, Wark, Shelbie, None), (2046854, Gib..."


### Salles par pays
#### Obseravations


In [ ]:
%%sql
select * from TitleDetail t where property="gross"

,id,title,production_year,kind,property,value
0,1640259,3 Ninjas: High Noon at Mega Mountain,1998,movie,gross,"$308,082 (USA) (19 April 1998)"
1,1640259,3 Ninjas: High Noon at Mega Mountain,1998,movie,gross,"$150,127 (USA) (12 April 1998)"
2,1640259,3 Ninjas: High Noon at Mega Mountain,1998,movie,gross,"$375,805 (USA)"
3,1640952,39 Pounds of Love,2005,movie,gross,"$28,125 (USA) (22 January 2006)"
4,1640952,39 Pounds of Love,2005,movie,gross,"$27,830 (USA) (15 January 2006)"
...,...,...,...,...,...,...
95,1810763,De-Lovely,2004,movie,gross,"€ 25,730 (Netherlands) (26 December 2004)"
96,1848399,Duet for One,1986,movie,gross,"$8,736 (USA) (28 December 1986)"
97,1848399,Duet for One,1986,movie,gross,"$8,736 (USA)"
98,1857778,El bosque animado,2001,movie,gross,"€ 379,762 (Italy) (30 March 2003)"


### Remarque
* le format de value pour la propriété "gross" ressemble à celui de la propriétés "weekend gross". Je vais vérifier que les deux ensembles ne sont pas égaux

In [ ]:
%%sql
select t.id from TitleDetail t where property="gross"
minus
select t.id from TitleDetail t where property="weekend gross"

,id
0,1640259
1,1658722
2,1666737
3,1714929
4,1723342
...,...
90,2361163
91,2497012
92,2511074
93,2512897


L'existence d'identifiants résultats à la requête ci-dessus prouve l'inégalité des deux ensembles

Je vais maintenant récupérer les informations concernant les pays ou les films sont sortis en salle et les revenus engendrés par pays

In [ ]:
%%sql 
-- extraction du nom du pays et du revenu associé
create or replace temp view Salles as 
select id, title, regexp_extract(split(value,'\\(')[1] , '(.+)\\)') as pays, 
                  replace(regexp_extract(split(value,'\\(')[0], '([0-9,]+)'), ',', '') as revenu
from TitleDetail where property="gross";

--Permet de récuperer le revenu d'un film en fonction du pays
create or replace temp view Revenu as 
select id, pays,sum(revenu) as revenu_total
from Salles
group by id,pays
order by revenu_total DESC;

-- regroupement des films sous forme de liste par id de film 
create or replace temp view SallesRevenu as 
select id, collect_set(pays) as paysSalles
from Revenu
group by id;

select * from SallesRevenu;

,id,paysSalles
0,1635380,"[Worldwide, UK, Spain, Singapore, Hong Kong, USA]"
1,1640259,[USA]
2,1640952,[USA]
3,1654054,[Hong Kong]
4,1654259,"[Germany, West Germany, UK, USA]"
...,...,...
95,2104845,[USA]
96,2105701,[Spain]
97,2111092,[USA]
98,2122913,[USA]


###Film (id, titre, roles, paysSalles) 

Jointure entre la vue contenant la liste des roles pour l'attribut roles et la vue contenant la liste des pays pour l'attribut paysSalles

In [ ]:
%%sql 
create or replace temp view Film as
select r.id, r.title, r.roles, s.paysSalles
from Roles r, SallesRevenu s
where r.id=s.id;

select * from Film;


,id,title,roles,paysSalles
0,1640259,3 Ninjas: High Noon at Mega Mountain,"[(602534, Grove, Mark Steven, None), (843014, ...",[USA]
1,1640952,39 Pounds of Love,"[(2914509, Medalia, Hilla, None), (3542921, Pe...",[USA]
2,1654259,A Nightmare on Elm Street: The Dream Child,"[(1475168, Spector, Craig, 1001), (2465675, Ri...","[Germany, West Germany, UK, USA]"
3,1658722,A Thin Line Between Love and Hate,"[(1827465, Bohanon, Greer, 17), (813686, King,...",[USA]
4,1666737,Adventure,"[(957233, Mahin, John Lee, None), (355305, Dav...",[USA]
...,...,...,...,...
95,2158608,Nocturno 29,"[(1833369, Bosé, Lucia, None), (193587, Brossa...",[Spain]
96,2162388,Nu ren si shi,"[(274166, Chiao, Roy, 2), (1183994, Pao, Fong,...",[Hong Kong]
97,2180078,Original Gangstas,"[(142055, Bill, Bushwick, 32), (1654913, Watso...",[USA]
98,2200270,Phantasm II,"[(2418718, Phillips, Samantha, 5), (318324, Co...","[Spain, USA]"


#### Question 7 : Fenêtres
Exprimer les requêtes :

 - 7a) Pour chaque mois, le revenu total des films sortis aux USA durant les 3 derniers mois. Le schéma du résultat est : 
     - (mois, année, revenuMoisCourant, revenu3DerniersMois)
 - 7b) Pour chaque pays, le classement des films par revenu total. Le schéma du résultat est :
      - (pays, id, titre, revenuTotal, classement)
 - 7c) Pour chaque film, la date de début et de fin des périodes où les revenus du week end représentent au moins 10% du revenu total annuel du film. Le schéma du résultat est :
     - (id, titre, revenuAnnuel, annee, mois, jour, revenuWeekEnd, numéroPériode)
       - Indications: Vous pouvez calculer la somme des revenus des différents pays en fixant un taux de conversion de devises. La condition 10% porte sur chaque revenu de week-end (pas de somme cumulée du revenu de la période). 


Verifier si tous les films ayant la property gross ont la property weekend gross

In [ ]:
%%sql
select t.id from TitleDetail t where t.property="weekend"
minus
select n1.id
from TitleDetail n1, TitleDetail n2
where n1.id=n2.id and n1.property="weekend gross" and n2.property="gross";

,id


Le résultat ci-dessus étant vide, selon confirme mon hypothèse. Les revenus avec la property weekend gross sont pris en compte dans gross


### Récupération de l'information de date
Pour calculer le revenu total des films sortis aux USA durant les 3 derniers mois, nous devons compléter la table Film avec les dates ( jour, mois, année ). Nous pouvons trouver cette information dans le champ value de la property gross.

### Table RevenuFilm
revenus de manière générale, car la question 7a) ne spécifie pas qu'il faut limiter le calcul aux weekends

In [ ]:
%%sql
create or replace temp view RevenuFilm as
Select distinct

      --id
      id,

      -- titre, 
      title as titre,

      --pays, 
      regexp_extract(value, '([^0-9]*) \\(([a-zA-Z ]*)\\)\\s', 2) as pays,

      --jour,
      regexp_extract(value, '(.*)\\(([0-9]{1,2})\\s([a-zA-Z]*)\\s([0-9]{4})',2) as jour,

      --mois, 
      regexp_extract(value, '(.*)\\(([0-9]{1,2})\\s([a-zA-Z]*)\\s([0-9]{4})',3) as mois,

      --année, 
      regexp_extract(value, '(.*)\\(([0-9]{1,2})\\s([a-zA-Z]*)\\s([0-9]{4})',4) as annee,

      --date : ajouter car il est plus simple de transformer le mois en entier depuis le type date
      to_date(regexp_extract(value,'(.*)\\(([0-9]{1,2}\\s[a-zA-Z]*\\s[0-9]{4})',2),'d MMMM yyyy') as date,

      --revenu : utiliser un regexp_replace afin de pouvoir transformer le string en int, la virgule empêchait cela
      regexp_replace(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\)', 2), ',', '') as revenu,

      --écrans : si on a pas les dernieres parenthèses, il n'y a pas d'écrans donc 0 sinon on retournes la valeur
      if(regexp_extract(value, '(.*)\\(([0-9]{1,2})\\s([a-zA-Z]*)\\s([0-9]{4})\\)([^0-9]*)([0-9,]*)',5)='',
         '0',
         regexp_extract(value, '(.*)\\(([0-9]{1,2})\\s([a-zA-Z]*)\\s([0-9]{4})\\)([^0-9]*)([0-9,]*)',6)) as ecrans

From TitleDetail
Where property = 'gross';

Select *
From RevenuFilm

,id,titre,pays,jour,mois,annee,date,revenu,ecrans
0,1640259,3 Ninjas: High Noon at Mega Mountain,USA,19,April,1998,1998-04-19,308082,0
1,1640259,3 Ninjas: High Noon at Mega Mountain,USA,12,April,1998,1998-04-12,150127,0
2,1640259,3 Ninjas: High Noon at Mega Mountain,,,,,None,375805,0
3,1654259,A Nightmare on Elm Street: The Dream Child,USA,29,October,1989,1989-10-29,22168359,0
4,1654259,A Nightmare on Elm Street: The Dream Child,USA,1,October,1989,1989-10-01,21374368,0
...,...,...,...,...,...,...,...,...,...
95,2303248,Source Code,,,,,None,2872410,0
96,2379933,The Imposter,USA,23,November,2012,2012-11-23,892409,0
97,2379933,The Imposter,USA,4,November,2012,2012-11-04,857661,0
98,2379933,The Imposter,USA,16,September,2012,2012-09-16,679130,0



### Vue avec les revenus du mois cours
 

In [ ]:
%%sql
create or replace temp view currentRev as 
Select annee, 
       month(date) as mois_num, 
       sum(revenu) as revenuMoisCourant
From RevenuFilm
Where pays = 'USA'
Group By annee, mois_num
Order By annee, mois_num;

Select * 
From currentRev;

,annee,mois_num,revenuMoisCourant
0,,NaN,135139959.0
1,1977,10.0,1837976.0
2,1982,1.0,3910032.0
3,1982,2.0,5568880.0
4,1984,2.0,58689.0
...,...,...,...
95,2007,4.0,26650991.0
96,2007,5.0,26709362.0
97,2007,6.0,35645819.0
98,2007,7.0,35695158.0


### requête avec fenêtrage sur cette vue sur l'attribut mois

In [ ]:
%%sql
create or replace temp view Revenu3DerniersMois as 
Select date, 
       revenu,
       sum(revenu) over (order by cast(date as timestamp) RANGE BETWEEN INTERVAL 3 MONTHS PRECEDING AND CURRENT ROW) as revenu3DerniersMois
From RevenuFilm
Where pays = 'USA'
Group By date, revenu
Order By date;

Select * 
From Revenu3DerniersMois

,date,revenu,revenu3DerniersMois
0,None,61363304,135139959.0
1,None,51264000,135139959.0
2,None,22512655,135139959.0
3,1977-10-23,1837976,1837976.0
4,1982-01-24,237811,237811.0
...,...,...,...
95,1997-06-29,61041533,701309973.0
96,1997-06-29,2770413,701309973.0
97,1997-07-06,2822376,761420639.0
98,1997-07-06,61191959,761420639.0


In [ ]:
%%sql

with T as (
Select year(date) as annee, 
       month(date) as mois, 
      last(revenu3DerniersMois) as revenu3DerniersMois
From Revenu3DerniersMois
Group By annee, mois
)

Select rmc.mois_num, rmc.annee, rmc.revenuMoisCourant, T.revenu3DerniersMois
From RevenuMoisCourant rmc
Left Join T On rmc.annee = T.annee and rmc.mois_num = T.mois
Order By rmc.annee, rmc.mois_num

,mois,annee,monthInt,revenuMoisCourant
0,April,1998,4,458209.0
1,December,2005,12,30113706.0
2,August,1996,8,165451674.0
3,March,1997,3,8646111.0
4,September,2004,9,140690669.0
...,...,...,...,...
95,May,1999,5,25407.0
96,July,2007,7,35695158.0
97,June,2005,6,23549865.0
98,May,2005,5,20438261.0


In [ ]:
%%sql

with T as (
Select year(date) as annee, 
       month(date) as mois, 
      last(revenu3DerniersMois) as revenu3DerniersMois
From Revenu3DerniersMois
Group By annee, mois
)

Select rmc.mois_num, rmc.annee, rmc.revenuMoisCourant, T.revenu3DerniersMois
From currentRev rmc
Left Join T On rmc.annee = T.annee and rmc.mois_num = T.mois
Order By rmc.annee, rmc.mois_num

,mois_num,annee,revenuMoisCourant,revenu3DerniersMois
0,NaN,,135139959.0,NaN
1,10.0,1977,1837976.0,1837976.0
2,1.0,1982,3910032.0,3910032.0
3,2.0,1982,5568880.0,9478912.0
4,2.0,1984,58689.0,58689.0
...,...,...,...,...
95,4.0,2007,26650991.0,133668189.0
96,5.0,2007,26709362.0,79896183.0
97,6.0,2007,35645819.0,89006172.0
98,7.0,2007,35695158.0,106945386.0


### 7b) Pour chaque pays, le classement des films par revenu total

* on utilise la fonction rank() qui permet d'avoir un ordre selon un attribut donnée en spécifiant l'attribut de partiion qui dans notre cas est le pays, et en spécifiant l'attribut sur lequel évaluer l'ordre du rang

In [ ]:
%%sql
with T as (
Select pays, 
       id, 
       titre, 
       sum(revenu) as revenuTotal
From RevenuFilm
Group By pays, id, titre
Order By pays, id, titre, sum(revenu) desc)

Select pays, id, titre, revenuTotal, rank() over (partition by pays order by revenuTotal desc) as classement
From T
Order By pays, classement

,pays,id,titre,revenuTotal,classement
0,,1714929,Basic Instinct,526303810.0,1
1,,1635380,102 Dalmatians,264788797.0,2
2,,2409929,The Saint,169500000.0,3
3,,2047411,La segunda guerra de los niños,147084643.0,4
4,,1788337,Coolie,130000000.0,5
...,...,...,...,...,...
95,,2398305,The October Brigade,836.0,96
96,Argentina,1890399,Final Destination 2,2273001.0,1
97,Argentina,1916458,Ghost Dog: The Way of the Samurai,1614667.0,2
98,Australia,1714929,Basic Instinct,12756984.0,1


#### 7c) Pour chaque film, la date de début et de fin des périodes où les revenus du week end représentent au moins 10% du revenu total annuel du film
## définition de la table RevenuWeekendFilm

In [ ]:
%%sql

create or replace temp view RevenuWeekEndFilm as
Select distinct

      --id
      id,

      -- titre, 
      title as titre,

      --pays, 
      regexp_extract(value, '([^0-9]*) \\(([a-zA-Z ]*)\\)\\s', 2) as pays,

      --jour,
      regexp_extract(value, '(.*)\\(([0-9]{1,2})\\s([a-zA-Z]*)\\s([0-9]{4})',2) as jour,

      --mois, 
      regexp_extract(value, '(.*)\\(([0-9]{1,2})\\s([a-zA-Z]*)\\s([0-9]{4})',3) as mois,

      --année, 
      regexp_extract(value, '(.*)\\(([0-9]{1,2})\\s([a-zA-Z]*)\\s([0-9]{4})',4) as annee,

      --date : ajouter car il est plus simple de transformer le mois en entier depuis le type date
      to_date(regexp_extract(value,'(.*)\\(([0-9]{1,2}\\s[a-zA-Z]*\\s[0-9]{4})',2),'d MMMM yyyy') as date,

      --revenu : utiliser un regexp_replace afin de pouvoir transformer le string en int, la virgule empêchait cela
      regexp_replace(regexp_extract(value, '([^0-9]*)([0-9,m]+) \\(([^\\)]*)\\)', 2), ',', '') as revenu,

      --écrans : si on a pas les dernieres parenthèses, il n'y a pas d'écrans donc 0 sinon on retournes la valeur
      if(regexp_extract(value, '(.*)\\(([0-9]{1,2})\\s([a-zA-Z]*)\\s([0-9]{4})\\)([^0-9]*)([0-9,]*)',5)='',
         '0',
         regexp_extract(value, '(.*)\\(([0-9]{1,2})\\s([a-zA-Z]*)\\s([0-9]{4})\\)([^0-9]*)([0-9,]*)',6)) as ecrans

From TitleDetail
Where property = 'weekend gross';

Select *
From RevenuWeekEndFilm

,id,titre,pays,jour,mois,annee,date,revenu,ecrans
0,1640952,39 Pounds of Love,USA,15,January,2006,2006-01-15,1227,1
1,1640952,39 Pounds of Love,USA,4,December,2005,2005-12-04,8055,1
2,1654259,A Nightmare on Elm Street: The Dream Child,USA,15,October,1989,1989-10-15,128659,233
3,1654259,A Nightmare on Elm Street: The Dream Child,USA,1,October,1989,1989-10-01,241955,306
4,1654259,A Nightmare on Elm Street: The Dream Child,USA,20,August,1989,1989-08-20,4084320,0
...,...,...,...,...,...,...,...,...,...
95,1674846,Aliens of the Deep,USA,21,January,2007,2007-01-21,6098,3
96,1674846,Aliens of the Deep,USA,7,January,2007,2007-01-07,6503,0
97,1674846,Aliens of the Deep,USA,31,December,2006,2006-12-31,15468,3
98,1674846,Aliens of the Deep,USA,24,December,2006,2006-12-24,3197,0


#### calcul du revenu annuel pour chaque film

In [ ]:
%%sql
create or replace temp view RevenuAnnuel as 
Select id, 
       titre, 
       annee, 
       sum(revenu) as revenuAnnuel
From RevenuWeekEndFilm
Group By id, titre, annee
Order By id, titre, annee;

Select * 
From RevenuAnnuel

,id,titre,annee,revenuAnnuel
0,1635380,102 Dalmatians,2000,43338350.0
1,1635380,102 Dalmatians,2001,12848606.0
2,1640952,39 Pounds of Love,2005,18964.0
3,1640952,39 Pounds of Love,2006,1317.0
4,1654259,A Nightmare on Elm Street: The Dream Child,1989,20398441.0
...,...,...,...,...
94,2509404,Xing xing wang,2000,612.0
95,2510177,Yaadein...,2001,43764673.0
96,2517945,Zatôichi,2003,87697.0
97,2517945,Zatôichi,2004,909273.0


### Revenu par weekend

In [ ]:
%%sql

create or replace temp view RevenuWeekEnd as 
Select id, 
       titre, 
       date, 
       sum(revenu) as revenuWeekEnd
From RevenuWeekEndFilm
Group By id, titre, date
Order By id, titre, date;

Select *
From RevenuWeekEnd

,id,titre,date,revenuWeekEnd
0,1635380,102 Dalmatians,2000-11-26,19883351.0
1,1635380,102 Dalmatians,2000-12-03,8295041.0
2,1635380,102 Dalmatians,2000-12-10,7412787.0
3,1635380,102 Dalmatians,2000-12-17,3621330.0
4,1635380,102 Dalmatians,2000-12-24,966854.0
...,...,...,...,...
95,1674846,Aliens of the Deep,2005-10-23,24331.0
96,1674846,Aliens of the Deep,2005-10-30,18520.0
97,1674846,Aliens of the Deep,2005-11-06,13780.0
98,1674846,Aliens of the Deep,2005-11-20,10435.0


### Ajout de la contrainte de 10%

In [ ]:
%%sql

create or replace temp view Revenu_jointure as 
Select a.id, 
       a.titre, 
       we.date, 
       a.annee, 
       we.revenuWeekEnd, 
       a.revenuAnnuel
From RevenuWeekEnd we
Left Join RevenuAnnuel as a On we.id = a.id and a.titre = we.titre and year(we.date) = annee
Where (we.revenuWeekEnd > a.revenuAnnuel*0.1)
Order By date;

Select * 
From Revenu_jointure

,id,titre,date,annee,revenuWeekEnd,revenuAnnuel
0,2082136,Looking for Mr. Goodbar,1977-10-23,1977,1540635.0,1540635.0
1,2477790,Vice Squad,1982-01-31,1982,1717205.0,3115730.0
2,2477790,Vice Squad,1982-02-07,1982,1160714.0,3115730.0
3,1791664,Crackers,1984-02-20,1984,58689.0,58689.0
4,2229370,Racing with the Moon,1984-03-25,1984,1803432.0,4245660.0
...,...,...,...,...,...,...
95,1911420,Garden State,2005-01-02,2005,74052.0,150030.0
96,1911420,Garden State,2005-01-09,2005,70391.0,150030.0
97,1674846,Aliens of the Deep,2005-01-30,2005,479368.0,4462468.0
98,1674846,Aliens of the Deep,2005-02-20,2005,515961.0,4462468.0


numérotation des périodes avec

In [ ]:
%%sql
Select id, 
       titre, 
       revenuAnnuel, 
       annee, 
       month(date) as mois, 
       day(date) as jour, 
       revenuWeekEnd,
       rank() over (partition by id order by date) as numeroPeriode
From Revenu_jointure

,id,titre,revenuAnnuel,annee,mois,jour,revenuWeekEnd,numeroPeriode
0,1635380,102 Dalmatians,43338350.0,2000,11,26,19883351.0,1
1,1635380,102 Dalmatians,43338350.0,2000,12,3,8295041.0,2
2,1635380,102 Dalmatians,43338350.0,2000,12,10,7412787.0,3
3,1635380,102 Dalmatians,12848606.0,2001,1,1,3472781.0,4
4,1635380,102 Dalmatians,12848606.0,2001,1,7,2639405.0,5
...,...,...,...,...,...,...,...,...
95,2005888,Julia,36382.0,2009,7,12,4688.0,4
96,2017777,Khosla Ka Ghosla!,15349.0,2006,9,24,12091.0,1
97,2017777,Khosla Ka Ghosla!,15349.0,2006,10,8,3258.0,2
98,2018134,Kickboxer 2: The Road Back,1120174.0,1991,6,16,444239.0,1
